# Putting a human face on machine learning

IBM Data Science Experience (DSX) is an interactive, collaborative, cloud-based environment where data scientists can use multiple tools to achieve insights.
Data Scientists can use the best of open source, tap into IBM's unique features, grow their skills, and collaborate with teams.
One of the many features of DSX provides the capability to create and train a machine learning model in DSX with little to no coding.
This model can subsequently be saved and deployed to Watson Machine Learning on IBM Bluemix and called for scoring in real-time. 

This notebook will demonstrate how to create a web front end to invoke the Titanic scoring model developed in prior lab steps.  

## Table of contents

1.  [Test model access by using a notebook](#test).
    - [1.1 Prerequisites](#test1).
    - [1.2 Retrieve your credentials](#test2).
    - [1.3 Authenticate and import libraries](#test3).   
2.  [Create a user interface](#create).
3.  [Publish app to Bluemix](#publish).
4.  [Summary and next steps](#summary).

<a id="test"></a>
## 1. Test model access by using a notebook. 

The first part of this notebook will confirm that the deployed model can be accessed via an external API. 
Please note that the credentials and endpoint used in this tutorial are only examples. 
You must substitute your own credentials and endpoint values. 

<a id="test1"></a>
### 1.1 Prerequisites.

This notebook and the corresponding tutorial sections require several IBM Bluemix services. 

- An Apache Spark instance which was created with your Data Science Experience account
- An Object Storage instance which was also created with your Data Science Experience account
- An IBM Watson Machine Learning instance which was created in the prior lab steps 
- A Python Flask instance which will be created later as part of this tutorial

<a id="test2"></a>
### 1.2 Retrieve your credentials.

To run the code samples in this notebook, you must supply information, such as the URL, username, password, and scoring endpoint variables from your environment. The details on how to do this are described in the following steps:

1. Log in to your **Bluemix** account(www.bluemix.net) and, from the dashboard, select the **IBM Watson Machine Learning** service that you created.
2. Click **Service credentials**.
3. Click **View credentials** for Credentials-1 and then copy the values from the **URL**, **username**, and **password** fields to the corresponding `URL`, `username`, and `password` variables in the code cells in this notebook below. 
4. To find the scoring endpoint, click **Manage** in the left panel.  
5. Click **Launch Dashboard** under **Watson Machine Learning**.  
6. Click on the **Deployments** tab. 
7. Click on the **Titanic** deployment. 
8. Scroll down to the API Details and copy the scoring endpoint url to the corresponding **scoring_endpoint** field in the code cell below. 


<a id="test3"></a>
### 1.3 Authenticate and import libraries.

After you update the following code cells with your specific information, you are ready to run the cells. Run the following cell to define your credentials to the Watson Machine Learning service and to import necessary libraries.

In [ ]:
# @hidden_cell
# Copy and paste url, username and password from Watson Machine Learning service credentials.
# Copy scoring_endpoint from Data Science Experience deployed model API details.
#url = 'enter_your_wml_instance_url'
#username = 'enter_your_username'
#password = 'enter_your_password'
#scoring_endpoint = 'enter_your_online_scoring_end_point'
url = ''
username = ''
password = ''
scoring_endpoint = ''
import urllib3, requests, json

To retrieve the token that you need to call the scoring API, run the following code:

In [ ]:
# Retrieves token to be used to call scoring API
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
path = '{}/v3/identity/token'.format(url)
response = requests.get(path, headers=headers)
mltoken = json.loads(response.text).get('token')

To call the scoring endpoint with some initial sample data, run the following code:

In [ ]:
# Call scoring endpoint with data payload
scoring_header = {'Content-Type': 'application/json', 'Authorization': 'Bearer' + mltoken}
payload = {"fields": ["pclass","name","sex","sibsp","parch","ticket","fare","embarked","Age_Bucket"], "values": [[1,"Jane Doe","female",0,2,"25",25,"S",2]]}
scoring = requests.post(scoring_endpoint, json=payload, headers=scoring_header)
scoringDICT = json.loads(scoring.text) 
scoringList = scoringDICT['values'].pop()[11:13]
    
score = scoringList[1:].pop()
probability_died = scoringList[0:1].pop()[0:1].pop()
probability_survived = scoringList[0:1].pop()[1:].pop()
if (score == 1.0) :
  score_str = "survived"
  probability = probability_survived                                        
else :
  score_str = "did not survive"
  probability = probability_died
print (score_str)
print (probability)

#print scoring.text

The result predicts that the passenger will survive with a probability of 94% (your result may vary slightly) given the input sample data.  This confirms that the deployed machine learning model can be called by using an API. 

<a id="create"></a>
## 2. Create a user interface 
After you confirm access to the API by running the code cells, it's time to start work on a basic Web application. For this example, you will create a user interface by using Python Flask. The following techniques can also be applied to other web app frameworks. For this tutorial, a boilerplate Python Flask web app will be created on Bluemix and the supporting files will be downloaded.  These files will be updated to integrate with your model and then pushed back up.

1. Start by deploying a Python Flask boilerplate. Sign on to your **Bluemix** account and click **Catalog**. On the left hand side click **Boilerplates** and then click > **Python Flask**. 
2. In the **App name** field, enter a unique name for your application. This name identifies your application and will appear in the URL that users need to access the app.
3. Click **Create**. Please give this some time to start.  The web app is running when there is a green dot next to the top title and it says **Running**.  To access this web app, click on the **Visit App URL** link.  The URL to access this Web application will be: `http://<your_app_name>.mybluemix.net` 
3. Download and unpack the command line interface and the starter code shown on this web page.  **Please spend some time on this page and become familiar with Cloud Foundry and Python Flask.** 
4. Now this boilerplate Python Flask Web application will be updated to call the deployed Titanic scoring model.  
5. In the folder where the unpacked starter code is located, create a file called `Titanic.py` and copy the following Python code into that file. As before, update the `url`, `username`, `password`, and `scoring_endpoint` fields to match with your deployed model. You may need to set pasting options to preserve the indentations which are critical for the following Python code. When completed, save this file. Note: For detailed information on coding in Flask, please refer to the following Web site: http://flask.pocoo.org 

```python
import urllib3, requests, json, os
from flask import Flask, render_template, request
from flask_bootstrap import Bootstrap
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, RadioField, FloatField, IntegerField
from wtforms.validators import Required, Length, NumberRange
url = ''
username = ''
password = ''
scoring_endpoint = '' 
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secretpassw0rd'
bootstrap = Bootstrap(app)
class TitanicForm(FlaskForm):
  pclass = RadioField('Passenger Class:', coerce=int, choices=[('1','First'),('2','Second'),('3','Third')])
  name = StringField('Name:')
  sex = RadioField('Gender:', coerce=str, choices=[('male','Male'),('female','Female')])
  age = RadioField('Age:', coerce=int, choices=[('0','0-5'),('1','6-11'),('2','12-17'),('3','18-39'),('4','40-64'),('5','65-79'),('6','>80')])
  ticket = StringField('Ticket:')
  fare = FloatField('Fare:')
  sibsp = IntegerField('Number of siblings/spouses:')
  parch = IntegerField('Number of parents/children:')
  embarked = RadioField('Embark Location:', coerce=str, choices=[('S','South Hampton'),('C','Cherbourg'),('Q','Queenstown')])
  submit = SubmitField('Submit')
@app.route('/', methods=['GET', 'POST'])
def index():
  form = TitanicForm()
  if form.is_submitted(): 
    sex = form.sex.data
    print (sex)
    form.sex.data = ''
    age = form.age.data
    print (age)
    form.age.data = ''
    name = form.name.data
    form.name.data = ''
    pclass = form.pclass.data
    form.pclass.data = ''
    ticket = form.ticket.data
    form.ticket.data = ''
    fare = form.fare.data
    form.fare.data = '' 
    sibsp = form.sibsp.data
    form.sibsp.data = ''
    parch = form.parch.data
    form.parch.data = ''   
    embarked = form.embarked.data
    form.embarked.data = ''    
    
    headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
    path = '{}/v3/identity/token'.format(url)
    response = requests.get(path, headers=headers)
    mltoken = json.loads(response.text).get('token')
    scoring_header = {'Content-Type': 'application/json', 'Authorization': 'Bearer' + mltoken}
    payload = {"fields": ["pclass","name","sex","sibsp","parch","ticket","fare","embarked","Age_Bucket"], "values": [[pclass,name,sex,sibsp,parch,ticket,fare,embarked,age]]}
    scoring = requests.post(scoring_endpoint, json=payload, headers=scoring_header)

    scoringDICT = json.loads(scoring.text) 
    scoringList = scoringDICT['values'].pop()[11:13]
    print (scoringList)
    score = scoringList[1:].pop()
    probability_died = scoringList[0:1].pop()[0:1].pop()
    print (probability_died)
    probability_survived = scoringList[0:1].pop()[1:].pop()
    if (score == 1.0) :
      score_str = "survived"
      probability = probability_survived                                        
    else :
      score_str = "did not survive"
      probability = probability_died
      
    
    return render_template('score.html', form=form, scoring=score_str,probability=probability)
  return render_template('index.html', form=form)
port = os.getenv('PORT', '5000')
if __name__ == "__main__":
  app.run(host='0.0.0.0', port=int(port))

```


### Setting up files

1.  Edit the `Procfile` file and replace the contents with the following line of code. 

    ```python
    web: python Titanic.py
    ```
    This points to the Python code to be executed that was created in the previous step. Save this file. 

2.  Add the following lines of code to the `requirements.txt` file. Flask should already be there. Save this file. 

    ```python
    Flask==0.10.1
    urllib3
    requests
    flask_bootstrap
    flask_wtf
    ```
3.  Two template web pages need to be added. Create a sub-folder called `templates`.  In the `templates` folder, create a file called `index.html` and copy the following code into the file. 

    ```python
    {% extends 'bootstrap/base.html' %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}Titanic Prediction{% endblock %}
{% block navbar %}
    <nav class="navbar navbar-inverse" role="navigation">
        <div class="container">
        <a class="navbar-brand" href="#">Titanic Prediction</a>
        </div>
    </nav>
{% endblock %}
{% block content %}
    <div class="container">
        <form method="POST" action="">
            <div class="row">
                <div class="col-md-12">
                <h3>To determine the survival prediction,please enter the following:</h3>
            </div>
        </div>
    <div class="row">
        <div class="col-md-8">
            <p>{{ form.pclass.label }} {{ wtf.form_field(form.pclass) }}</p>
            <p>{{ form.name.label }} {{ form.name }}</p>
            <p>{{ form.sex.label }} {{ wtf.form_field(form.sex) }}</p>
	        <p>{{ form.sibsp.label }} {{ form.sibsp }}</p>
            <p>{{ form.parch.label }} {{ form.parch }}</p>	           
            <p>{{ form.ticket.label }} {{ form.ticket }}</p>
            <p>{{ form.fare.label }} {{ form.fare }}</p>
            <p>{{ form.embarked.label }} {{ wtf.form_field(form.embarked) }}</p>
	        <p>{{ form.age.label }} {{ wtf.form_field(form.age) }}</p>
            <br>
            {{ form.submit() }}
            {{ form.hidden_tag() }}
        </div>
    </div>
</div>
{% endblock %}
    ```

4.  Save this file. 

5.  In the `templates` folder, create a file called `score.html` and copy the following lines of code into the file you create. 

    ```python
    {% extends 'bootstrap/base.html' %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}Titanic Prediction{% endblock %}
{% block navbar %}
    <nav class="navbar navbar-inverse" role="navigation">
        <div class="container">
        <a class="navbar-brand" href="/">Titanic Prediction</a>
        </div>
    </nav>
{% endblock %}
{% block content %}
    <div class="container">
        <div class="row">
            <div class="col-md-12">
                prediction:{{ scoring }} 
			</div>
			<div class="col-md-12">
			    probability: {{ probability }}
		    </div>
        </div>
        <div class="row">
            <div class="col-md-12">
            <h4><a href="/">Try Again!</h4>
            </div>
        </div>
    </div>
{% endblock %}
    ```
    
6.  Save this file.

<a id="publish"></a>
## Step 3: Publish

You can now publish this updated web application to Bluemix. 
 
1.  Go to the folder where the starter code is located. **Hint**: If you’re in the `templates` folder, move up one folder. 
2.  Type `bx login` and press **Enter**.  You are prompted for your email id.  Type it in and press **Enter**. 
3.  Type your password and press **Enter**. 
4.  If prompted, type the number that corresponds to your organization and press **Enter**.
5.  Type `bx cf push` and press **Enter**. If successful, the responses will display the **App started** message and the **OK** status as well as information about the instances and usage.  Please give this some time to complete.
6.  Once started, point your browser to the web application URL. The URL consists of the name that you used for the app `your_app_name`, inserted into the following URL: http://your_app_name.mybluemix.net  
7. 	When the page renders, provide the input and then click **Submit**. If you see the boilerplate Web page, you must refresh the browser page. 

The web page displays whether the passenger is predicted to survive and the probability. To observe probabilities for other variations, click **Try Again!**.  It should be noted that a skilled web developer can take this raw data and present it elegantly.

<a id="summary"></a>
## 4. Summary and next steps

Congratulations! You have completed this tutorial that demonstrates how a deployed model in IBM Watson Machine Learning can be called in real time for scoring. By using  advanced web application development skills, the user interface can be significantly enhanced and other web app technologies leveraged. 


## Author

This example is based on a notebook developed by the **Daniel Kikuchi** 

**Daniel Kikuchi** is an Executive IT Specialist and a North America Big Data Technical Sales Specialist in the IBM Analytics organization. He is passionate about developing integrated solutions that address client business needs. Dan has often been tapped by IBM upper management to lead critical, complex and challenging client engagements. In his free time, Dan enjoys archery and is a Certified Level Two Archery Instructor with the US Archery & National Field Archery Association.


Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.